In [ ]:
import psycopg2

SCHEMA = 'historical'
CONN_DICT = {
    'host': 'main-us-e2.cmbsiiqeauby.us-east-2.rds.amazonaws.com',
    'user': 'doadmin',
    'password': 'i39kew8n7jcat7l9',
    'port': 5432,
    'dbname': 'main'
}


def connect_to_db():
    return psycopg2.connect(**CONN_DICT)

def get_cur(conn):
    conn.rollback()
    return conn.cursor()

conn = connect_to_db()

In [ ]:
from datetime import datetime, timedelta
t365 = datetime.now() - timedelta(days=365)
date_str = t365.strftime('%Y-%m-%d')

query = f"""
select match_id 
from historical_csgo.csgo_match_maps
where map_id not in (
	SELECT DISTINCT(map_id)
	FROM historical_csgo.csgo_map_rounds cmr 
	WHERE round_number != "t_score" + "ct_score"
	ORDER BY 1
) AND map_id NOT IN (
	SELECT DISTINCT(map_id) 
	FROM historical_csgo.csgo_map_rounds cmr
	GROUP BY 1
	HAVING max(round_number) < 16
	ORDER BY 1
) AND map_id NOT IN (
	SELECT distinct(map_id)
	FROM historical_csgo.csgo_match_maps cmm
	WHERE cmm.winner IS NULL OR cmm.winner = ''
) AND match_id not in (
	SELECT match_id
	FROM historical_csgo.csgo_match_data cmd
	WHERE cmd.date < '{date_str}'
)
"""
cur = get_cur(conn)
cur.execute(query)
match_ids = [x[0] for x in cur.fetchall()]
match_ids_str = ','.join([str(x) for x in match_ids])
print(f'Found {len(match_ids)} working. sample: {match_ids[:5]}')
cur.close()

In [ ]:
import pandas as pd
import numpy as np
import time

In [ ]:
from query_reader import QueryReader
qr = QueryReader('queries')
raw = qr.read_query('econ_rebuild.sql')
raw = raw.format(date="'2021-05-01'")
df = pd.read_sql(raw, conn)
df

In [ ]:
def streak(df, column_name, ref_value):
    df = df.sort_values(['map_id', 'steam_id', 'round_number'])
    prev_steam_id = df.iloc[0]['steam_id']

    df[column_name] = 0 
    prev_value = 0

    def func2(row):
        # non local variable ==> will use pre_value from the new_fun function
        nonlocal prev_value
        nonlocal prev_steam_id
        if prev_steam_id != row['steam_id']:
            # Since this is for larger dfs, we have to set the streak to one, because when we call apply we ommit the first row.
            # What if instead we just returned 0?
            prev_steam_id = row['steam_id']
            prev_value = 1
            return 1

        if row['won'] == ref_value:
            new_value =  prev_value + 1
        else:
            new_value = 0 

        prev_value = new_value
        return new_value

    # This line might throw a SettingWithCopyWarning warning
    df[column_name] = df.apply(func2, axis=1)
    return df

# This needs to be split changed because it currently takes the whole column and assigns the whole column it does not account for 
# multiple players in the DF
df = streak(df, 'win_streak', 1)
df = streak(df, 'loss_streak', 0)
df[['map_id', 'round_number', 'steam_id', 'won', 'is_tside', 'win_streak', 'loss_streak']].head(50)

In [ ]:
ct_reasons = ['BombDefused', 'CTWin', 'TargetSaved']
t_reasons = ['TerroristsWin', 'TargetBombed']

df['new_money'] = 0
df['new_money'] = df['round_number'].apply(lambda x: 0 if x not in (1,16) else 800)
df

In [ ]:
# Need to activiate if a team wins two rounds, and then start counting loss bonuses

losing_bonus = {
    1: 1400,
    2: 1900,
    3: 2400,
    4: 2900,
    5: 3400
    # 5+ is 3400
}

after_2_wins_loss_bonus = {
    0: 1400,
    1: 1500,
    2: 2000,
    3: 2500,
    4: 3000
    # 4+ is 3000
}

BOMB_PLANT_BONUS = 300
PLAYER_BOMB_DEFUSE_BONUS = 300
TEAM_BOMB_DEFUSE_BONUS = 250
ROUND_WIN_BASE = 3250
# Round End conditions
# Enemy team elimnated: 3250
# CT Wins by time: 3250
# Bomb Defused: 3500
# Bomb Exploded: 3500

# bomb planted or defused = 300

In [ ]:
gdf = df.groupby(['match_id', 'map_id', 'round_number']).max()[['bomb_defused', 'bomb_exploded']]
gdf = gdf.reset_index()
print(gdf.shape)
mdf = df.merge(gdf, on=['match_id', 'map_id', 'round_number'])
mdf['team_bomb_defused'] = mdf['bomb_defused_y']
del mdf['bomb_defused_y']
mdf['team_bomb_exploded'] = mdf['bomb_exploded_y']
del mdf['bomb_exploded_y']

mdf = mdf.sort_values(['steam_id', 'round_number'])
mdf['bomb_plant_bonus'] = mdf.apply(lambda x: BOMB_PLANT_BONUS if x['bomb_plants'] and x['is_tside'] else 0, axis=1) 
mdf['bomb_defuse_bonus'] = mdf.apply(lambda x: PLAYER_BOMB_DEFUSE_BONUS if x['bomb_defused_x'] and not x['is_tside'] else 0, axis=1) 
mdf['team_bomb_defuse_bonus'] = mdf.apply(lambda x: TEAM_BOMB_DEFUSE_BONUS if not x['is_tside'] and x['team_bomb_defused'] else 0, axis=1 )
mdf['team_bomb_explode_bonus'] = mdf.apply(lambda x: TEAM_BOMB_DEFUSE_BONUS if x['is_tside'] and x['team_bomb_exploded'] else 0, axis=1 )
mdf.head(25)

In [ ]:
mdf

In [ ]:
def round_end_bonus(df):
    df = df.sort_values(['steam_id', 'round_number'])
    previous_row = df.iloc[0]
    previous_steam_id = df.iloc[0]['steam_id']

    latest_win_streak_after_loss = df.iloc[0]['win_streak']
    df['round_end_bonus'] = 0

    def inner(row):
        nonlocal previous_row
        nonlocal latest_win_streak_after_loss

        if row['steam_id'] != previous_steam_id:
            latest_win_streak_after_loss = 0 

        if row['won'] == 1:
            latest_win_streak_after_loss = 0 
            round_end_bonus = ROUND_WIN_BASE
        else:
            # Check if team has won at least two consecutive rounds
            if previous_row['win_streak'] >= 2 or latest_win_streak_after_loss >= 2:
                if latest_win_streak_after_loss <= previous_row['win_streak']:
                    latest_win_streak_after_loss = previous_row['win_streak'] 

                if latest_win_streak_after_loss >= 4 or row['loss_streak'] >= 4:
                    round_end_bonus = after_2_wins_loss_bonus[4]
                else:
                    round_end_bonus = after_2_wins_loss_bonus[row['loss_streak']]
            else:
                if row['loss_streak'] >= 5:
                    round_end_bonus = losing_bonus[5] 
                else:
                    try:
                        round_end_bonus = losing_bonus[row['loss_streak']]
                    except Exception as e:
                        print('prev row')
                        print(previous_row)
                        print('')
                        print('')
                        print('current')
                        print(row)
                        raise e



        row['round_end_bonus'] = round_end_bonus
        previous_row = row.copy()
        return row

    df = df.apply(inner, axis=1)
    return df

mdf = round_end_bonus(mdf)
    


In [ ]:
mdf[['round_number', 'steam_id', 'won', 'is_tside', 'win_streak', 'loss_streak', 'round_end_bonus']].head(25)

In [ ]:
weapon_money = {
    'Pistol': 300,
    'Knife': 1500,
    'Cz75': 100,
    'SMG': 600,
    'P90': 300,
    'Shotgun': 900,
    'Rifle': 300,
    'Sniper': 300,
    'Machine Gun': 300,
    'AWP': 100,
    'Grenade': 300,
    'Zeus': 0
}
mdf['ar_bonus'] = mdf['rifle_kills'].apply(lambda x: x * weapon_money['Rifle']) 
mdf['sniper_bonus'] = mdf['sniper_kills'].apply(lambda x: x*weapon_money['Sniper'])
mdf['grenade_bonus'] = mdf['grenade_kills'].apply(lambda x: x*weapon_money['Grenade'])
mdf['lmg_bonus'] = mdf['lmg_kills'].apply(lambda x: x*weapon_money['Machine Gun'])
mdf['shotgun_bonus'] = mdf['shotgun_kills'].apply(lambda x: x*weapon_money['Shotgun'])
mdf['smg_bonus'] = mdf['smg_kills'].apply(lambda x: x*weapon_money['SMG'])
mdf['pistol_bonus'] = mdf['pistol_kills'].apply(lambda x: x*weapon_money['Pistol'])
mdf['melee_bonus'] = mdf['melee_kills'].apply(lambda x: x*weapon_money['Knife']) 
mdf['awp_bonus'] = mdf['awp_kills'].apply(lambda x: x*weapon_money['AWP'])
mdf['p90_bonus'] = mdf['p90_kills'].apply(lambda x: x*weapon_money['P90'])
mdf['cz75_bonus'] = mdf['cz75_kills'].apply(lambda x: x*weapon_money['Cz75'])

In [ ]:
mdf['total_money_made'] = mdf[[x for x in mdf.columns if x.endswith('_bonus')]].sum(axis=1)
# mdf[['match_id', 'map_id', 'round_number', 'steam_id', 'total_money_made', 'parser_money']]

In [ ]:
mdf[mdf['map_id']==14908][[x for x in mdf.columns if x.endswith('_bonus')]]

In [ ]:
money_df = mdf.groupby(['match_id', 'map_id', 'steam_id']).sum()['total_money_made']
money_df = money_df.reset_index()
money_df

In [ ]:
print('shape', money_df.shape)
match_money_df = money_df.groupby(['match_id', 'steam_id']).agg(total_money_per_map=('total_money_made', 'mean'), total_money=('total_money_made','sum')).reset_index()
match_money_df.to_csv('csgo_money.csv',index=False)
print('Shape', match_money_df.shape)
match_money_df.head()

In [ ]:
money_df.shape